In [1]:
%matplotlib inline

# So it's not necessary to restart the kernel after updating any other file
%load_ext autoreload
%autoreload 2

#import matplotlib.pyplot as plt

In [2]:
from BNReasoner import BNReasoner
from BayesNet import BayesNet
import networkx as nx

net = BayesNet()
bn = BNReasoner(net)

net.load_from_bifxml('testing/lecture_example.BIFXML')
net.get_all_variables()
graph = net.get_interaction_graph()
#net.draw_structure()

In [3]:
# Sets XSet and Yset are independent given EvSet 
# # iff every path between a variable in XSet and a variable in YSet 
# are blocked by a variable in EvSet

def d_sep_indep(G,XSet,EvSet,YSet):
    # Find all undirected paths between vars of XSet and YSet
    for X in XSet:
        for Y in YSet:
            for path in nx.all_shortest_paths(G, X, Y):
                # Only if ALL paths are blocked, there is independence
                #if blocked(path, EvSet) == False:
                 #   return False
                print(block(path,X,Y,EvSet))
                print(len(path))    
    return True


def block (p, X,Y,Z):
    # sequential valve
    for step in range(0,len(p)-2):
        if (p[step+1] in net.get_children(p[step]) and p[step+2] in net.get_children(p[step+1])):
            if p[step+1] not in Z:
                return False
        #converging valve
        if p[step+1] in net.get_children(p[step]) and p[step+1] in net.get_children(p[step+2]):
            if p[step+1] in Z or nx.descendants(graph,p[step+1]) in Z:
                return False
        #diverging valve
        if p[step] in net.get_children(p[step+1]) and p[step+2] in net.get_children(p[step+1]):
            return False 
    

# Check if path is blocked by some variable in EvSet
def blocked(p, evlist):
    ev_on_path = intersection(p, evlist)
    # If intersectionSet is not empty, the path is blocked
    if len(ev_on_path):
        return True
    else: 
        return False
        
# Return intersectionset of two lists        
def intersection(A, B):
    return set(A).intersection(B)



In [4]:
#check 1: winter is not independent of wet grass given JUST rain
print(d_sep_indep(graph,['Winter?'],['Rain?'],['Wet Grass?']))

#check 2: winter is independent of wet grass given rain AND sprinkler
print(d_sep_indep(graph,['Winter?'],['Rain?','Sprinkler?'],['Wet Grass?']))

#check 3: Winter is independent of slippery road given rain 
print(d_sep_indep(graph,['Winter?'],['Rain?'],['Slippery Road?']))

#check 4: Winter is NOT independent of Slippery road given sprinkler and wet grass
print(d_sep_indep(graph,['Winter?'],['Sprinkler?','Wet Grass?'],['Slippery Road?']))



False
3
None
3
True
None
3
None
3
True
None
3
True
False
3
True


In [26]:
1 * 0.1*0.5*0.9

0.045000000000000005

In [27]:
1 * (0.1*0.5*0.9)

0.045000000000000005

In [31]:
%%time
bn.joint_probability(sort=True)

Wall time: 11 ms


,Winter?,Sprinkler?,Rain?,Wet Grass?,Slippery Road?,p
0,True,True,True,True,True,0.06384
8,True,True,True,True,False,0.02736
2,True,True,True,False,True,0.00336
10,True,True,True,False,False,0.00144
16,True,True,False,True,True,0.00000
24,True,True,False,True,False,0.02160
18,True,True,False,False,True,0.00000
26,True,True,False,False,False,0.00240
4,True,False,True,True,True,0.21504
12,True,False,True,True,False,0.09216


In [36]:
%%time
bn.marginal_distribution(
    Q=["Wet Grass?","Slippery Road?"],
    E={"Winter?": True},
    normalize=True,
    sort=True
)

Wall time: 16 ms


[   Sprinkler?  Rain?  Wet Grass?     p
 7        True   True        True  0.95
 6        True   True       False  0.05
 3        True  False        True  0.90
 2        True  False       False  0.10
 5       False   True        True  0.80
 4       False   True       False  0.20
 1       False  False        True  0.00
 0       False  False       False  1.00,
    Rain?  Slippery Road?    p
 3   True            True  0.7
 2   True           False  0.3
 1  False            True  0.0
 0  False           False  1.0]